In [1]:
import pandas as pd
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from scipy import stats

from sklearn import preprocessing

%matplotlib inline

from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error
from sklearn.metrics import median_absolute_error, mean_squared_error, mean_squared_log_error

from scipy.optimize import minimize
import statsmodels.tsa.api as smt
import statsmodels.api as sm

from tqdm import tqdm_notebook

from itertools import product

def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

import warnings
warnings.filterwarnings('ignore')

sns.set(color_codes=True)

In [4]:
Google_trends_df = pd.read_csv("./New Data/Google_trends_data_updated.csv",index_col=0)



In [5]:
comp_list = [   
 ['CSCO', 'Cisco'],
 ['T', 'AT&T'],
 ['BAC', 'Bank of America'],
 ['ADBE', 'Adobe'],
 ['PEP', 'Pepsi'],
 ['LLY', 'Eli Lilly'],
 ['MRK', 'Merck'],
 ['ORCL', 'Oracle'],
 ['PG', 'Procter & Gamble'],
 ['AMZN', 'Amazon'],
 ['INTC', 'Intel'],
 ['KO', 'CocaCola'],
 ['MSFT', 'Microsoft'],
 ['NVDA', 'NVIDIA'],
 ['HD', 'The Home Depot'],
 ['XOM', 'Exxon'],
 ['CVX', 'Chevron'],
 ['CMCSA', 'Comcast'],
 ['FB', 'Facebook'],
 ['DIS', 'Disney'],
 ['NFLX', 'Netflix'],
 ['JPM', 'JPMorgan Chase'],
 ['ABT', 'Abbott Laboratories'],
 ['GOOGL', 'Google'],
 ['PFE', 'Pfizer'],
 ['VZ', 'Verizon'],
 ['UNH', 'UnitedHealth Group'],
 ['AAPL', 'Apple'],
 ['WMT', 'Walmart'],
 ['JNJ', 'Johnson & Johnson']   
]

In [6]:
# Stock data of all companies
for df_name, company in comp_list:
    globals()[df_name] = pd.read_csv("./Stock Data/" + df_name + ".csv",index_col=0)
    globals()[df_name]['company_name'] = company
    
    
# #     DatetimeIndex, diff, standardizing
    globals()[df_name].Date = pd.to_datetime(globals()[df_name].Date)
    globals()[df_name] = globals()[df_name].set_index('Date')
    globals()[df_name] = globals()[df_name][['company_name', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'day_of_week']]
    globals()[df_name].columns = ['company_name', 'Open', 'High', 'Low', 'Close', 'Adj_Close', 'Volume',
       'day_of_week']
        
    #diff value each day
    globals()[df_name]['Open_diff']=globals()[df_name].Open.diff()
    globals()[df_name]['Close_diff']=globals()[df_name].Close.diff()
    globals()[df_name]['Adj_Close_diff']=globals()[df_name].Adj_Close.diff()
    globals()[df_name]['Volume_diff'] = globals()[df_name].Volume.diff()
    
    globals()[df_name] = pd.concat([globals()[df_name],
                                    Google_trends_df[company],
                                    Google_trends_df['Covid-19'],
                                    Google_trends_df['Pandemic'],
                                    Google_trends_df['presidential election'],
                                    Google_trends_df['Lockdown'],
                                    Google_trends_df['Face masks'],
                                    Google_trends_df['Coronavirus'],
                                    Google_trends_df['Quarantine'],
                                    Google_trends_df['EDD'],
                                    Google_trends_df['sba loans'],
                                    Google_trends_df['unemployment check'],
                                    Google_trends_df['swine flu'],
                                    Google_trends_df['flu'],
                                    Google_trends_df['zika']],axis=1)

#Changing column names
    globals()[df_name].columns = ['company_name', 'Open', 'High', 'Low', 'Close', 'Adj_Close', 'Volume',
       'day_of_week', 'Open_diff', 'Close_diff', 'Adj_Close_diff',
       'Volume_diff', 'company_Google_trends', 'Covid_19', 'Pandemic', 'presidential_election',
       'Lockdown', 'Face_masks', 'Coronavirus', 'Quarantine', 'EDD',
       'sba_loans', 'unemployment_check', 'swine_flu','flu','zika']
    
    
    
    # standardizing
    a = globals()[df_name][['Open','Open_diff', 
                              'High', 'Low', 'Close','Close_diff', 
                              'Adj_Close','Adj_Close_diff', 
                              'Volume','Volume_diff',
                              'company_Google_trends', 
                              'Covid_19', 'Pandemic',
                              'presidential_election', 'Lockdown', 
                              'Face_masks', 'Coronavirus','Quarantine', 
                              'EDD', 'sba_loans', 'unemployment_check', 'swine_flu','flu','zika']]
    
    mean = np.mean(a, axis=0)
    std = np.std(a, axis=0, ddof=1)
    a_std = (a-mean)/std
    
    
    globals()[df_name] = pd.concat([globals()[df_name]['company_name'],globals()[df_name]['day_of_week'],a_std],axis=1)
    
    
    #Changing column orders
    globals()[df_name]= globals()[df_name][['company_name', 'Open','Open_diff', 
                              'High', 'Low', 'Close','Close_diff', 
                              'Adj_Close','Adj_Close_diff', 
                              'Volume','Volume_diff',
                              'day_of_week', 'company_Google_trends', 
                              'Covid_19', 'Pandemic',
                              'presidential_election', 'Lockdown', 
                              'Face_masks', 'Coronavirus','Quarantine', 
                              'EDD', 'sba_loans', 'unemployment_check', 'swine_flu','flu','zika']]
    
    
    



In [9]:
All = pd.concat([CSCO, T, BAC, ADBE, PEP, LLY, MRK, ORCL, PG, AMZN, INTC, KO, MSFT, NVDA, HD, XOM, CVX, CMCSA, FB, DIS, NFLX, JPM, ABT, GOOGL, PFE, VZ, UNH, AAPL, WMT, JNJ])

All

,company_name,Open,Open_diff,High,Low,Close,Close_diff,Adj_Close,Adj_Close_diff,Volume,...,Lockdown,Face_masks,Coronavirus,Quarantine,EDD,sba_loans,unemployment_check,swine_flu,flu,zika
2011-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.098940,-0.198693,-0.130474,-0.099610,-1.202383,0.403785,0.048305,0.531745,-0.136513,-0.433880
2011-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.196800,-0.172584,-0.130474,-0.150262,-0.684407,-0.138576,0.042157,0.414138,-0.024419,-0.433880
2011-01-03,Cisco,-0.883667,NaN,-0.885036,-0.874727,-0.881563,NaN,-0.938823,NaN,0.906280,...,-0.110453,-0.229549,-0.130474,-0.148963,0.299748,0.496383,0.214303,0.453341,0.174861,-0.433880
2011-01-04,Cisco,-0.878216,0.134197,-0.886837,-0.872892,-0.878836,0.055294,-0.936897,0.037926,0.560244,...,-0.104696,-0.241416,-0.130474,-0.164549,0.273850,-0.231174,0.165119,0.845361,0.100131,-0.433880
2011-01-05,Cisco,-0.871855,0.159389,-0.864323,-0.860964,-0.856110,0.577646,-0.920845,0.467735,1.555182,...,-0.001079,-0.277019,-0.130474,-0.134677,0.196153,0.417013,0.238896,0.610149,0.112586,-0.433880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04-26,Johnson & Johnson,1.886068,-0.028991,1.880852,1.878188,1.862496,-0.025766,1.990197,-0.031917,-0.018831,...,3.585210,7.994742,2.751313,8.524223,8.000329,1.065200,7.503884,1.600654,1.267500,-0.278490
2020-04-27,Johnson & Johnson,1.871932,-0.451543,1.876242,1.884945,1.851892,-0.311709,1.980831,-0.334222,0.039003,...,4.460195,7.520035,2.931425,8.098227,10.244893,3.710861,10.168055,1.438619,1.121437,-0.267391
2020-04-28,Johnson & Johnson,1.739496,-3.987579,1.767458,1.761062,1.743994,-2.935321,1.885527,-3.107958,-0.039274,...,4.132076,8.706801,2.751313,7.672230,15.079338,2.652597,8.733501,1.519637,1.170125,-0.253517
2020-04-29,Johnson & Johnson,1.744333,0.115569,1.713250,1.703250,1.701207,-1.179551,1.847734,-1.251722,0.116400,...,3.585210,8.706801,2.661257,7.246233,10.935528,2.388030,8.938437,1.276584,1.364875,-0.264616


In [10]:
All.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 102240 entries, 2011-01-01 to 2020-04-30
Data columns (total 26 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   company_name           101649 non-null  object 
 1   Open                   101649 non-null  float64
 2   Open_diff              101619 non-null  float64
 3   High                   101649 non-null  float64
 4   Low                    101649 non-null  float64
 5   Close                  101649 non-null  float64
 6   Close_diff             101619 non-null  float64
 7   Adj_Close              101649 non-null  float64
 8   Adj_Close_diff         101619 non-null  float64
 9   Volume                 101649 non-null  float64
 10  Volume_diff            101619 non-null  float64
 11  day_of_week            101649 non-null  object 
 12  company_Google_trends  102240 non-null  float64
 13  Covid_19               102240 non-null  float64
 14  Pandemic            

In [12]:
All = All.reset_index()

# Date_date
All['Date_date']=All['index'].apply(lambda x:x.date())
All = All.sort_values(by='index')
All = All.set_index('index')
All = All['2011-01-03':'2020-04-29']

In [13]:
All.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 102150 entries, 2011-01-03 to 2020-04-29
Data columns (total 27 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   company_name           101649 non-null  object 
 1   Open                   101649 non-null  float64
 2   Open_diff              101619 non-null  float64
 3   High                   101649 non-null  float64
 4   Low                    101649 non-null  float64
 5   Close                  101649 non-null  float64
 6   Close_diff             101619 non-null  float64
 7   Adj_Close              101649 non-null  float64
 8   Adj_Close_diff         101619 non-null  float64
 9   Volume                 101649 non-null  float64
 10  Volume_diff            101619 non-null  float64
 11  day_of_week            101649 non-null  object 
 12  company_Google_trends  102150 non-null  float64
 13  Covid_19               102150 non-null  float64
 14  Pandemic            

In [14]:
All.to_csv("All.csv", encoding = "utf-8")